### Import python library

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

### 1. Read data


In [2]:
timestep = 3

In [3]:
PCT_lstm_train_X = pd.read_csv("AVG_lstm_final_train_X33.csv")
PCT_lstm_train_y = pd.read_csv("AVG_lstm_final_train_y33.csv")

PCT_lstm_test_X = pd.read_csv("AVG_lstm_final_test_X33.csv")
PCT_lstm_test_y = pd.read_csv("AVG_lstm_final_test_y33.csv")

In [4]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [5]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

#### 4) input shape로 변경 (row, timestep=2, feature)

ex) 
timestep = 2

* X_train_v 구성예시: [[1 ~ 24경기 데이터, 25 ~ 48경기 데이터], [49 ~ 72경기 데이터, 73 ~ 96경기 데이터] ]  
X_train_v.shape >> (2,2*x)             # x: 각 24group에 대한 변수 개수
* y_train_v 구성예시: 97 ~ 120 경기 승률

=> reshape

* X_train_v.shape >> (2,2,x)  # row, timestep, feature

### 2. 모델 구성(LSTM)
- optimizer: Adam -> lr(learning rate) 조절
- LSTM: 모델이 계속 동일한 결과값이 나올 때, input 뉴런 개수를 늘려야 한다는 글을 읽고 계속 input 노드 개수를 바꿔주면서 모델 생성중
- loss: MSE

- early_stop: patience를 크게하면 과적합 되는 경우가 있어서 최대한 작게 설정해둠
- batch_size: 모델이 계속 동일한 결과값이 나올 때, 데이터가 적어 batch size를 줄여보라는 글을 읽고 1로 설정해둠

In [6]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred',"shift_1","shift_2","shift_3",'rms','rms_avg'])

idx = 0
for y in year:
    tmp1 = PCT_lstm_train_X[PCT_lstm_train_X["YEAR"] == y]
    tmp2 = PCT_lstm_train_y[PCT_lstm_train_y["YEAR"] == y]
    tmp3 = PCT_lstm_test_X[PCT_lstm_test_X["YEAR"] == y]
    tmp4 = PCT_lstm_test_y[PCT_lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        
        X_train_t = X_train_v.reshape(X_train_v.shape[0], timestep,X_train_v.shape[1]//timestep)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], timestep,X_test_v.shape[1]//timestep)
        
        
        ## model
        K.clear_session() 

        model = Sequential()
        optimizer = Adam(lr=0.01)
#         optimizer = RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)

        model.add(LSTM(100,input_shape = (timestep,X_train_v.shape[1]//timestep))) # (timestep, feature)
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

        model.summary()
        
#         hist1 = model.fit(X_train_t, y_train_v, epochs=100, batch_size=1, verbose=1)
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train_t, y_train_v, epochs=100,
                  batch_size=1, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test_t)
        rms = sqrt(mean_squared_error(y_test_v, y_pred))
        rms05 = sqrt(mean_squared_error(y_test_v,[y_train.mean()[0]]))
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0],
                                  X_test.loc[X_test.index[0],["shift_AVG_1"]][0],
                                  X_test.loc[X_test.index[0],["shift_AVG_2"]][0],
                                X_test.loc[X_test.index[0],["shift_AVG_3"]][0],
                                rms,rms05]

        idx += 1

test_pred_df[['y','y_pred',"shift_1","shift_2","shift_3",'rms','rms_avg']] = test_pred_df[['y','y_pred',"shift_1","shift_2","shift_3",'rms','rms_avg']].astype(float)

2016LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58800     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,901
Trainable params: 58,901
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 2s 88ms/sample - loss: 0.9200 - mae: 0.5369
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0023 - mae: 0.0428
Epoch 3/100
25/25 [==============================] - 0s 4ms/sample - loss: 3.1170e-04 - mae: 0.0148
Epoch 4/100
25/25 [==============================] - 0s 4ms/sample - loss: 2.0717e-04 - mae: 0.0117
Epoch 5/100
25/25 [==============================] - 0

25/25 [==============================] - 0s 4ms/sample - loss: 5.8044e-05 - mae: 0.0062
Epoch 10/100
25/25 [==============================] - 0s 4ms/sample - loss: 4.9405e-05 - mae: 0.0056
Epoch 00010: early stopping
2016KT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58800     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,901
Trainable params: 58,901
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 2s 75ms/sample - loss: 0.6381 - mae: 0.4409
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0016 - mae: 0.0358
Epoch 3/100
25/25 [==================

25/25 [==============================] - 2s 80ms/sample - loss: 0.7626 - mae: 0.5321
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0061 - mae: 0.0664
Epoch 3/100
25/25 [==============================] - 0s 4ms/sample - loss: 5.8057e-04 - mae: 0.0201
Epoch 4/100
25/25 [==============================] - 0s 4ms/sample - loss: 2.7006e-04 - mae: 0.0145
Epoch 5/100
25/25 [==============================] - 0s 5ms/sample - loss: 2.3640e-04 - mae: 0.0128
Epoch 6/100
25/25 [==============================] - 0s 5ms/sample - loss: 2.1861e-04 - mae: 0.0133
Epoch 7/100
25/25 [==============================] - 0s 4ms/sample - loss: 3.0464e-04 - mae: 0.0152
Epoch 8/100
25/25 [==============================] - 0s 4ms/sample - loss: 4.1320e-04 - mae: 0.0179
Epoch 00008: early stopping
2017HH =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Par

25/25 [==============================] - 2s 97ms/sample - loss: 0.7218 - mae: 0.4493
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0016 - mae: 0.0332
Epoch 3/100
25/25 [==============================] - 0s 3ms/sample - loss: 2.0277e-04 - mae: 0.0118
Epoch 4/100
25/25 [==============================] - 0s 3ms/sample - loss: 9.0450e-05 - mae: 0.0080
Epoch 5/100
25/25 [==============================] - 0s 3ms/sample - loss: 7.7072e-05 - mae: 0.0070
Epoch 6/100
25/25 [==============================] - 0s 3ms/sample - loss: 6.8702e-05 - mae: 0.0071
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 7.9585e-05 - mae: 0.0076
Epoch 8/100
25/25 [==============================] - 0s 3ms/sample - loss: 1.1642e-04 - mae: 0.0084
Epoch 00008: early stopping
2017LT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Par

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58800     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,901
Trainable params: 58,901
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 2s 85ms/sample - loss: 0.8976 - mae: 0.5266
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0042 - mae: 0.0542
Epoch 3/100
25/25 [==============================] - 0s 4ms/sample - loss: 5.2846e-04 - mae: 0.0177
Epoch 4/100
25/25 [==============================] - 0s 4ms/sample - loss: 1.3186e-04 - mae: 0.0088
Epoch 5/100
25/25 [==============================] - 0s 4ms/sample - loss: 1.6219e-04 - mae: 0.0108
E

25/25 [==============================] - 0s 4ms/sample - loss: 2.0318e-04 - mae: 0.0119
Epoch 16/100
25/25 [==============================] - 0s 4ms/sample - loss: 8.2686e-05 - mae: 0.0069
Epoch 17/100
25/25 [==============================] - 0s 4ms/sample - loss: 7.3809e-05 - mae: 0.0074
Epoch 18/100
25/25 [==============================] - 0s 4ms/sample - loss: 6.5586e-05 - mae: 0.0065
Epoch 19/100
25/25 [==============================] - 0s 5ms/sample - loss: 8.3589e-05 - mae: 0.0080
Epoch 20/100
25/25 [==============================] - 0s 6ms/sample - loss: 8.0645e-05 - mae: 0.0073
Epoch 00020: early stopping
2018WO =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58800     
_________________________________________________________________
dense (Dense)                (None, 1)          

25/25 [==============================] - 0s 3ms/sample - loss: 2.9365e-04 - mae: 0.0141
Epoch 5/100
25/25 [==============================] - 0s 3ms/sample - loss: 2.2250e-04 - mae: 0.0125
Epoch 6/100
25/25 [==============================] - 0s 3ms/sample - loss: 2.2341e-04 - mae: 0.0124
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 1.9333e-04 - mae: 0.0115
Epoch 8/100
25/25 [==============================] - 0s 3ms/sample - loss: 2.7418e-04 - mae: 0.0140
Epoch 9/100
25/25 [==============================] - 0s 4ms/sample - loss: 2.1298e-04 - mae: 0.0116
Epoch 00009: early stopping
2019HH =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58800     
_________________________________________________________________
dense (Dense)                (None, 1)               

25/25 [==============================] - 0s 3ms/sample - loss: 1.3617e-04 - mae: 0.0087
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 1.0465e-04 - mae: 0.0079
Epoch 00007: early stopping
2019KT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58800     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,901
Trainable params: 58,901
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 2s 82ms/sample - loss: 0.6137 - mae: 0.4605
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0023 - mae: 0.0398
Epoch 3/100
25/25 [===================

25/25 [==============================] - 0s 4ms/sample - loss: 1.8489e-04 - mae: 0.0112
Epoch 7/100
25/25 [==============================] - 0s 4ms/sample - loss: 1.8797e-04 - mae: 0.0117
Epoch 00007: early stopping


In [8]:
# test_pred_df.sort_values(by=["YEAR","T_ID"]).to_csv("AVG_t2.csv",index = False)

In [7]:
mean_squared_error(test_pred_df['y'],test_pred_df['y_pred'])

0.0007636969319700142

In [25]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_1,shift_2,shift_3,rms,rms_avg
YEAR,,,,,,,
2016,0.291146,0.293826,0.292375,0.296354,0.286139,0.016245,0.020702
2017,0.291601,0.285180,0.284698,0.293109,0.292798,0.024749,0.017521
2018,0.285146,0.290292,0.293297,0.283993,0.287721,0.013357,0.012211
2019,0.257812,0.270008,0.269043,0.272767,0.264472,0.025187,0.018150


In [26]:
test_pred_df

,YEAR,T_ID,y,y_pred,shift_1,shift_2,shift_3,rms,rms_avg
0,2016,LG,0.296069,0.301487,0.294471,0.297110,0.305842,0.005418,0.012246
1,2016,HH,0.288575,0.302395,0.293083,0.314581,0.284884,0.013820,0.017133
2,2016,NC,0.287440,0.294829,0.294611,0.286241,0.294798,0.007389,0.006483
3,2016,HT,0.256739,0.298921,0.293286,0.313860,0.274251,0.042182,0.046513
4,2016,SK,0.305263,0.297950,0.286055,0.308046,0.303571,0.007313,0.006729
5,2016,KT,0.295455,0.273820,0.281437,0.262626,0.253911,0.021635,0.028792
6,2016,WO,0.289941,0.294226,0.297398,0.326291,0.290284,0.004285,0.014889
7,2016,LT,0.309893,0.273439,0.256250,0.285211,0.290888,0.036454,0.045589
8,2016,SS,0.283863,0.305396,0.329186,0.283688,0.277251,0.021533,0.025611
9,2016,OB,0.298225,0.295802,0.297974,0.285885,0.285714,0.002423,0.003036
